# <center>Lightwave Explorer
<center>
Nick Karpowicz
</center>
<center>
Max Planck Institute of Quantum optics
</center>
<center>
<img src="Images/LWEicon.png" height="256" width="256">
</center>
<center>
(icon made by Stable Diffusion)
</center>

## What and why

Lightwave explorer is an open source nonlinear optics simulator, intended to be fast, visual, and flexible for students and researchers to play with ultrashort laser pulses and nonlinear optics without having to buy a laser first.

The simulation itself is written in CUDA in order to run on graphics cards. This makes it fast enough to give almost instant feedback to the user, which can hopefully make it easier to develop that nebulous but important "feeling" for how things fit together. 

Although its target platform is the GPU, I've written it in a "bilingual" pattern of CUDA so that exactly the same code can be compiled as c++ to run on the CPU. A CPU will not be nearly as fast as a GPU, about an order of magnitude in my experience. However, since one of the goals of this project was for the results we publish to be fully transparent and reproducible, I think it's important that as many people as possible have access to it.

### Main goals:
 - _Easily extensible database of materials:_ Eveything the program knows about nonlinear materials comes from a human-readable text file giving the appropriate coefficients and tensors. If you want to use a new material, or you've done a measurement in a new range where typical extrapolations from older data isn't relevant, it's easy to add and correct. There are places for references for the key parameters, and these references are stored in the saved simulation results for future reference. Especially if you have simulations that you checked against experiments, I'd be very happy for you to add your crystal definitions to the central database in the project Github.
 - _Efficient code so that complicated systems can be simulated in 3D:_ Real laser pulses can be messy, and if they weren't so before a nonlinear crystal, there's a good chance they are after (but not always). If things are slow, it's hard to go beyond one dimension on tolerable time scales, and then you miss out on the whole weird world of spatiotemporal couplings. Here you have options for rather fast simulations when there's a symmetry to apply (e.g. cylindrical or along one Cartesian dimension), alongside fully 3D propagation. Runs natively on both GPU and CPU to make use of whatever you have to work with.
 - _A graphical interface that lets you see what you're doing:_ A lot of us think in visual terms. Being able to adjust and scan parameters and immediately see what happens can really make it easier to understand what you're looking at. 
 - _A flexible sequence mode:_ By stringing together elements, not just nonlinear crystals but also spherical or parabolic mirrors, apertures, filters, free space propagation and other elements, simulate how  one interaction affects another. Sequences of events can be scripted and even programmed with loop functions to see how things change over the course of repeated interactions.
 - _Fitting modes:_ Sometimes the data that we measure depends on an interesting way on a parameter, and we'd actually like to go back and figure out what that parameter was from the data. Solving this kind of inverse problem can be tough when the parameter lives inside a partial differential equation, but by simulating the whole thing and doing a fit, you have a chance to do it! The fitting algorithm can be used to narrow down a huge space of variables to come at your best estimation of what was happening in an experiment, or to adjust your experimental system to maximize output at a given frequency.
 - _A Python module for easy postprocessing of the results:_ I hope that you get something interesting out that you want to plot and maybe publish. One of the nicest platforms for making nice plots is Python in my opinion (that's why this documentation is in a Jupyter notebook), so purely out of self interest I tried to make it easy to load the results in Python. The module also has some functions related to typical operations you'd like to do on the data to make it easy for all of us. The program also gives you a Matlab loading script for those who want to use that.
 - _Command line interface for running on Linux/clusters:_ The main application runs on Windows, but the simulation core can be compiled on Linux. I use this to run it on the clusters of the Max Planck Society, and other institutes and universities likely have similar systems. This lets you do a lot more if your personal resources are limited but you want to run simulations on a large grid or cover a lot of different parameters!


Physical model
=====

Basic equations
-----

I'm sure we've all derived the wave equation from Maxwell's equations before, so I'm not going to repeat that. Let's just talk about the equations I'm using and where they come from.

In the final version there will be two main modes, one based on the wave equation in the paraxial approximation, where we make a simplification based on the assumption that the light is moving primarily along a given direction, and the full solution of Maxwell's equations on a grid. Right now the latter isn't available yet! But in the vast majority of cases, the former is what you want to use.

The nonlinear, unidirectional wave equation:

\begin{equation}
\frac{\partial}{\partial z} E(\mathbf{x} ,\omega) = i(k_z + \frac{1}{2k_z}\nabla^2_\perp)E(\mathbf{x} ,\omega) + \frac{i\omega}{2\epsilon_0 \tilde{n}\left(\omega\right)c}\mathbf{P}^{\mathrm{NL}}\left(\mathbf{x},\omega\right)
\end{equation}

where propagation of the field is assumed to be primarily along the $z$ axis. $\nabla^2_\perp$ is the transverse Laplacian (_e.g._ $\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}$ in Cartesian coordinates). $\omega$ is the angular frequency (this equation is in the frequency domain), $\mathbf{P}^{NL}$ is the nonlinear polarization, $\tilde{n}$ is the complex index of refraction, and the other constants have their usual definition. Bold $\mathbf{x}$ is shorthand for either $x$, $y$, and $z$ in Cartesian coordinates, or $\rho$, $\theta$, and $z$ in cylindrical coordinates.

In deriving this equation, we make the assumption that $\left|k_z^2\right| \gg \left|\frac{\partial^2}{\partial z^2} \tilde{A}(z)\right|$, where $\tilde{A}$ is the complex amplitude of a given frequency component of the electric field. This means that we are assuming that the field evolves gradually on the spatial scale of the wavelength. This is a good assumption when the light is propagating in the presence of "normal" nonlinearities, where the nonlinear signal builds up slowly over time. It's not a good assumption near interfaces, where the reflection from the surface suddenly changes the evolution of the wave, and there is a counterpropagting wave. So, for thin crystals, or surface nonlinearities, we'll want to fully solve Maxwell's equations (which will be implemented later). You'll notice that the spatial form of this equation is pretty similar to the Helmholtz equation. The underlying assumptions are quite similar.

If $\mathbf{P}^{NL} = 0$, we have linear propagation of light, and for example a Gaussian beam is a solution to the above equation. When it's nonzero is when things get interesting. We'll go over how that happens in a bit.

Linear crystal properties
-----
For every material we're going to deal with, we need to have a way of describing its refractive index. This is typically done with a parameterized equation known generally as a Sellmeier equation. There are many different forms of this that you'll find in literature. So far, I've only implemented two of them here:

### General fitting sellmeier equation
$\begin{equation}
n^2 = a[0] + \frac{a[1] + a[2]}{\lambda^2 + a[3]} + \frac{a[4] + a[5]}{\lambda^2 + a[6]} + \frac{a[7] + a[8]}{\lambda^2 + a[9]} + \frac{a[10] + a[11]}{\lambda^2 + a[12]} + a[13] \lambda^2 + a[14] \lambda^4 + a[15] \lambda^6 + \frac{k a[16]}{a[17] - \omega^2 + i a[18]\omega} + \frac{k a[19]}{a[20] - \omega^2 + i a[21]\omega} \tag{2}
\end{equation}$

There are quite a few terms. You're not expected to use all of them, they're just there to make it more likely that it can accommodate any equation you find in literature, containing common forms of various elements. The constant $k = \frac{e^2}{\epsilon_0 m_e}$ has a value of about 3182.6 in SI units. The last two terms are complex valued; this allows you to add absorption, with a Lorentzian line shape.

Let's look at a concrete example.

In [1]:
#importing some modules I'll want
import lightwaveExplorer as lwe
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
#from matplotlib_inline.backend_inline import set_matplotlib_formats
#set_matplotlib_formats('pdf', 'svg')
from matplotlib import rcParams
rcParams['font.family'] = 'Arial'

#first we'll make a wavelength grid to work with
l = np.linspace(0.3,3,1024)

#next we'll need Sellmeier coefficients, these are for barium fluoride, H. H. Li., J. Phys. Chem. Ref. Data 9, 161-289 (1980)
a = np.array([1.33973,0,0.81070,-0.010130,0,0.19652,-892.22,0,4.52469,-2896.6,0,0,1,0,0,0,0,0,0,0,0,0])

#we can get the refractive index for the wavelengths we put in the grid by calling the sellmeier() function
#from the lightwaveExplorer module, with the equationType set to 0.
n = lwe.sellmeier(l, a, 0)

#let's make it so we can adjust the resonances of the oscillators and see how it affects the index
def adjustBaF2Index(uv: float, IR: float):
    fig,ax = plt.subplots(1,1, figsize=(5,4))
    a2 = np.array([1.33973,0,0.81070,-0.010130,0,0.19652,-892.22,0,4.52469,-2896.6,0,0,1,0,0,0,0,0,0,0,0,0])
    a2[3] *= uv
    a2[6] *= IR
    n2 = lwe.sellmeier(l, a2, 0)
    ax.plot(l,np.real(n),label="original",color="blue")
    ax.plot(l,np.real(n2), label = "modified", color = "magenta")
    ax.set_xlabel("Wavelength (microns)")
    ax.set_ylabel("n")
    ax.legend()

    
interPlot = widgets.interactive(adjustBaF2Index, uv=widgets.FloatSlider(min=0.5, max=2, step=0.1, value=1), IR=widgets.FloatSlider(min=0.1, max=2, step=0.1, value=1));
interPlot

interactive(children=(FloatSlider(value=1.0, description='uv', max=2.0, min=0.5), FloatSlider(value=1.0, descr…

You can play with the position of the resonances and see how this affects the value of the refractive index, and its dispersion. Feel free to add sliders or attach them to other parameters to see what they do. Using the function above, you can use a fitting routine to get coefficients in the appropriate format if there isn't a 1-to-1 mapping from literature to the equation.

The other equation that's implemented in the code is all Lorentzians:

$\begin{equation}
n^2 = \frac{k a[0]}{a[1] - \omega^2 + i a[2]\omega} + \frac{k a[3]}{a[4] - \omega^2 + i a[5]\omega} + ... \tag{3}
\end{equation}$
all the way up to $a[20]$.

This one will be useful when we solve Maxwell's equation on a grid, because there it will be important to obey causality. This latter equation does, while the previous equation (although more flexible) does not.

Nonlinear crystal properties
-----

There are currently three different types of nonlinear effects that can happen inside of the simulation: second order, such as second harmonic generation and difference frequency generation, third order, such as the Kerr effect, four-wave mixing, and third harmonic generation, and multiphoton absorption/nonlinear plasma interaction in a classical picture of free carriers.

Pulse parameters
-----

Numerical model
=====

Propagation scheme, convergence, stability
-----

Cylindrical symmetry tricks
-----

When the beam can be assumed to be symmetric around the z-axis, we can simplify things (and speed up the simulation) a lot, since the grid can be reduced from 3 dimensions, $(x, y, t)$, to 2, $(\rho, t)$. 

The only thing that makes life more difficult after this coordinate transformation is the transverse Laplacian from eqn. (1). In cylindrical coordinates with radial symmetry, it becomes

$\begin{equation}
\nabla^2_\perp = \frac{\partial^2}{\partial \rho^2} + \frac{1}{\rho} \frac{\partial}{\partial \rho} \tag{4}
\end{equation}$

Compared to a Cartesian coordinate, we have an extra term to deal with. And unfortunately, unlike Cartesian coordinates, that term is not a diagonal operator in the Fourier domain.

One approach is to make a Hankel transform instead of a Fourier transform, in which case the operator will be diagonal. Numerically this can be implemented in a fairly efficient way leveraging FFTs, but only if the data is arranged on a logarithmic grid.

Another way is to use an evenly spaced grid, but explicitly calculate the $\frac{1}{\rho}$ term in real space. This way one uses FFTs and the same grid as the calculations in Cartesian coordinates, but with the expense of having to do a Fourier transform at each propagation step. Since we have to do that anyway for the inclusion of nonlinearities, it's not a big deal in this case.

The grid used in this case extends to the negative side of the origin, so that the FFT doesn't see a discontinuity at $\rho = 0$, covering the range $-\rho_{\textrm{max}}$ to $+\rho_{\textrm{max}}$. This would seem to be a waste of memory since the positive and negative sides of the origin are identical by symmetry.

If we use a grid that is truly symmetric, that is indeed the case: we would be solving everything at $\rho = ... -3, -2, -1, 0, +1, +2, +3...$ etc.

However, what if we shift the placement of our grid points by $\frac{1}{4}$? Now we have the fields at $\rho = ... -2\frac{3}{4}, -1\frac{3}{4}, -\frac{3}{4}, +\frac{1}{4}, +1\frac{1}{4}, +2\frac{1}{4}, +3\frac{1}{4}$

This way, the positive and negative sides of the grid have different information: the previously-useless negative-$\rho$ values contain the midpoints between the points on the positive $\rho$ grid. The spacing is still 1 however, meaning that the grid doesn't include high values of the transverse $k$, which often bring numerical instability, in order to get doubled resolution in space. We just have to "interlace" the grid using our knowledge of the symmetry of the system. 

We make use of this in two places in the cylindrical symmetry propagation mode: to eliminate _aliasing_ in the calculation of the nonlinear polarization, and to improve the accuracy of the finite-difference calculation of the radial derivative included in that $\frac{1}{\rho} \frac{\partial}{\partial \rho}$ term of the Laplacian.

This results in overall easier convergence of calculations in cylindrical symmetry mode, which is why it's a good place to start when approaching a new system.

Simulation structure
-----

Using sequences
-----

Fitting mode
-----

File formats
-----

Python interface
-----